In [1]:
import tensorflow as tf 

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [3]:
data_x, data_y = [], []
num_id = 9
img_length = 500*500*3

for i in range(0, num_id):
    folder = 'pixiv/id_'+str(i)
    for img_file in os.listdir(folder):
#         print(folder+'/'+img_file)
        img = np.array(Image.open(folder+'/'+img_file))
        img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
        if(len(img)>img_length):
            img = np.array([img[j] for j in range(len(img)) if j%4!=3])
            print(len(img))
#         data_x.append([j/255.0 for j in img])
        data_x.append(img/255.0)
        
        label = [0 for j in range(0, num_id)]
        label[i]=1
        data_y.append(label)   
    print(folder+' is completed.')

print(len(data_x))
print(len(data_y))

750000
pixiv/id_0 is completed.
pixiv/id_1 is completed.
pixiv/id_2 is completed.
pixiv/id_3 is completed.
pixiv/id_4 is completed.
pixiv/id_5 is completed.
pixiv/id_6 is completed.
pixiv/id_7 is completed.
750000
750000
750000
750000
750000
750000
750000
pixiv/id_8 is completed.
1800
1800


In [4]:
data_x = np.array(data_x, dtype='float32')
data_y = np.array(data_y, dtype='float32')

print(data_x[:10])
print(data_y[:10])

[[ 0.78823531  0.77254903  0.87843138 ...,  0.99607843  0.99607843
   0.99607843]
 [ 0.78039217  0.86274511  0.92941177 ...,  0.98039216  0.98039216
   0.98039216]
 [ 0.79607844  0.91764706  0.85882354 ...,  0.63529414  0.63137257
   0.56078434]
 ..., 
 [ 0.99607843  0.99607843  0.99607843 ...,  0.86274511  0.7764706
   0.54901963]
 [ 1.          0.89019608  0.7019608  ...,  0.63529414  0.25490198
   0.1882353 ]
 [ 0.13333334  0.13333334  0.13333334 ...,  0.13333334  0.13333334
   0.13333334]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [5]:
order = [i for i in range(len(data_x))]
random.shuffle(order)

print(len(order))
n=int(len(data_x)*9/10)
print(n)

train_x = [data_x[i] for i in order[0:n]]
train_y = [data_y[i] for i in order[0:n]]

test_x = [data_x[i] for i in order[n:]]
test_y = [data_y[i] for i in order[n:]]

print(len(train_x))
print(len(test_y))

1800
1620
1620
180


In [8]:
graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    labels = tf.placeholder('float', [None, num_id])            #
    inputs = tf.placeholder('float', [None, img_length])                        #输入的数据占位符
#     print(x.shape)
#     print(y_actual.shape)

    def weight_variable(shape): #定义一个函数，用于初始化所有的权值 W
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape): #定义一个函数，用于初始化所有的偏置项 b
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W): #定义一个函数，用于构建卷积层
        return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='VALID')

    def max_pool(x): #定义一个函数，用于构建池化层
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

    #构建网络
    x_image = tf.reshape(inputs, [-1,500,500,3])         #转换输入数据shape,以便于用于网络中
    W_conv1 = weight_variable([5, 5, 3, 32])      
    b_conv1 = bias_variable([32])       
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)     #第一个卷积层
    h_pool1 = max_pool(h_conv1)                                  #第一个池化层
    print(h_pool1.shape)#(?, 124, 124, 32)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)      #第二个卷积层
    h_pool2 = max_pool(h_conv2)                                   #第二个池化层
    print(h_pool2.shape)#(?, 30, 30, 64)

    # 第三层 是个全连接层,输入维数125*125*64, 输出维数为1024
    W_fc1 = weight_variable([30 * 30 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 30*30*64])              #reshape成向量
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)    #第一个全连接层

    keep_prob = tf.placeholder("float") 
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)                  #dropout层

    W_fc2 = weight_variable([1024, num_id])
    b_fc2 = bias_variable([num_id])
#     y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   #softmax层
    y_predict=tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#     loss = -tf.reduce_sum(labels*tf.log(y_predict))     #交叉熵
#     optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)    #梯度下降法
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels = labels))
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
    
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(labels,1))    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))    #精确度计算

(?, 124, 124, 32)
(?, 30, 30, 64)


In [27]:
batch_start = 0
batch_size = 50
training_steps = 41

with tf.Session(graph=graph) as session:
#     saver.restore(session, tf.train.latest_checkpoint('lstm_check'))
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(training_steps): 
        if(batch_start+batch_size>=len(train_x)):
            order = [i for i in range(len(train_x))]
            random.shuffle(order)
            train_x = [train_x[i] for i in order]
            train_y = [train_y[i] for i in order]
            batch_start = 0        
        feed_x = train_x[batch_start:batch_start+batch_size]
        feed_y = train_y[batch_start:batch_start+batch_size]
#         print(feed_x.shape)
#         print(feed_y.shape)
        feed_dict = {inputs: feed_x, labels: feed_y, keep_prob:0.6}
        _, l, predictions, accur = session.run([optimizer, loss, labels, accuracy], feed_dict = feed_dict)
        if (step % 10 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('accuarcy is ', accur)
#             saver.save(session, 'lstm_check/my-model-1.ckpt', global_step=step+1000)
    test_loss, test_logits, test_accur = session.run([loss,labels,accuracy], feed_dict={inputs: test_x, labels: test_y, keep_prob:1.0})

Initialized
Loss at step 0: 57.009102
accuarcy is  0.08
Loss at step 10: 2.115653
accuarcy is  0.24
Loss at step 20: 1.561283
accuarcy is  0.42
Loss at step 30: 1.105225
accuarcy is  0.64
Loss at step 40: 0.479545
accuarcy is  0.84


In [28]:
print('test loss, ', test_loss, ' test accur is ', test_accur)

test loss,  24.3489  test accur is  0.216667
